# DEFINE PARAMETERS (this is the only thing to modify if you just want to train a model)

In [1]:
# Roboflow parameters
roboflow_api_key = "MJQUATZvpcKoBxRjLuXx"
roboflow_workspace_name = "auv2024"
roboflow_project_name = "front-cam-felps"
roboflow_project_version = 5

# Training parameters
target_classes = ["Buoy", "Gate"]
train_test_val_split = (0.7, 0.2, 0.1)
model_save_filename = "best_AUV_sim_front_cam_model.pt"
epoch_increments = 1 # save the model weights to google drive every `epoch_increments` epochs
batch_size = 16

# Custom augmentation parameters
colorAugmentProb = 0.5
noiseAugmentProb = 0.5
resolutionAugmentProb = 0.5
contrastAugmentProb = 0.5
blurAugmentProb = 0.5
brightnessAugmentProb = 0.0

# WARNING: do not change these "randomly", check the YoloV8 docs for what these parameters affect before modifying (these values have worked well)
# See the last cell for where these parameters are used in the model
degrees = 360
flipud = 0.5
fliplr = 0.5
max_perspective_change = 0.001
max_translate = 0.1
max_scale_change = 0.3
mosaic = 0.5
mixup = 0.5

# Implementation

## Setup Python dependencies 

In [2]:
!pip install roboflow
!pip install albumentations
!pip install opencv-python
!pip install ultralytics

  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_amd64.whl.metadata (19 kB)
Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_amd64.whl (38.0 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.4 requires opencv-python-headless>=4.9.0, but you have opencv-python-headless 4.8.0.74 which is incompatible.


  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.0.74
    Uninstalling opencv-python-headless-4.8.0.74:
      Successfully uninstalled opencv-python-headless-4.8.0.74


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.28 requires opencv-python-headless==4.8.0.74, but you have opencv-python-headless 4.9.0.80 which is incompatible.


In [3]:
!mkdir data
!mkdir data\augmented
!mkdir data\augmented\train
!mkdir data\augmented\test
!mkdir data\augmented\val
!mkdir data\augmented\train\images
!mkdir data\augmented\test\images
!mkdir data\augmented\val\images
!mkdir data\augmented\train\labels
!mkdir data\augmented\test\labels
!mkdir data\augmented\val\labels
!mkdir data\raw
!mkdir data\raw\images
!mkdir data\raw\labels

In [4]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
from roboflow import Roboflow

## Define YOLO classes

In [5]:
data_folder_absolute_path = os.path.abspath("data")
print("Data absolute path:", data_folder_absolute_path)

with open('data.yaml', 'w+') as f:
    f.write("train: {}\\augmented\\train\\images\n".format(data_folder_absolute_path))
    f.write("test: {}\\augmented\\test\\images\n".format(data_folder_absolute_path))
    f.write("val: {}\\augmented\\val\\images\n".format(data_folder_absolute_path))
    f.write("nc: {}\n".format(len(target_classes)))
    f.write('names: {}'.format(target_classes))

Data absolute path: c:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data


## Define augmentation functions

In [6]:
#given a list of samples, make two copies of each sample that are darker/brighter to simulate differently lit environments
def brightnessAugment(images):
    out = []
    for image in images:
        transform = A.Compose([ A.augmentations.transforms.ColorJitter (brightness=(1.05, 1.05), contrast=0, saturation=0, hue=0, always_apply=True) ])
        bright_img = transform(image=image)["image"]
        transform = A.Compose([ A.augmentations.transforms.ColorJitter (brightness=(0.95, 0.95), contrast=0, saturation=0, hue=0, always_apply=True) ])
        dark_img = transform(image=image)["image"]
        out.append(bright_img)
        out.append(dark_img)
    return out

#given a list of samples, make a copy of each sample but more blurred to simulate objects out of focus, dirty lenses, and backscattering
def blurAugment(images):
    out = []
    for image in images:
        ksize = (10, 10) # lower to lower blur
        blurred_img = cv2.blur(image, ksize)
        out.append(blurred_img)
    return out

#given a list of samples, make a copy of each sample but with a lower contrast image to simulate backscattering and over/under-exposure
def contrastAugment(images):
    out = []
    for image in images:
        transform = A.Compose([ A.augmentations.transforms.ColorJitter (brightness=0, contrast=(0.1, 0.1), saturation=0, hue=0, always_apply=True) ])
        decontrasted_img = transform(image=image)["image"]
        out.append(decontrasted_img)
    return out

#given a list of samples, make a copy of each sample but with camera noise added to the image to simulate different camera feeds
def noiseAugment(images):
    out = []
    for image in images:
        transform = A.Compose([ A.augmentations.transforms.ISONoise(color_shift=(0.01, 0.01), intensity=(0.8, 0.8), always_apply=True) ])
        noisy_img = transform(image=image)["image"]
        out.append(noisy_img)
    return out

#given a list of samples, make a copy of each sample but with the image downscaled (lower resolution of image) to simulate lower quality cameras/images
def resolutionAugment(images):
    out = []
    for image in images:
        #interpolation=A.augmentations.transforms.Interpolation(downscale=cv2.INTER_NEAREST, upscale=cv2.INTER_NEAREST)
        transform = A.Compose([ A.augmentations.transforms.Downscale(scale_min=0.25, scale_max=0.25, always_apply=True) ])
        low_res_img = transform(image=image)["image"]
        out.append(low_res_img)
    return out

#increase intensity of blues in given image
def make_bluer(img, color_shift_intensity):
    img_b, img_g, img_r = cv2.split(img) #split by channel
    img_b = np.uint16(img_b)
    img_b += color_shift_intensity
    np.clip(img_b, 0, 255, out=img_b)
    img_b = np.uint8(img_b)
    img = cv2.merge((img_b, img_g, img_r)) #merge adjusted channels
    del img_b
    del img_g
    del img_r
    return img

#increase intensity of greens in given image
def make_greener(img, color_shift_intensity):
    img_b, img_g, img_r = cv2.split(img) #split by channel
    img_g = np.uint16(img_g)
    img_g += color_shift_intensity
    np.clip(img_g, 0, 255, out=img_g)
    img_g = np.uint8(img_g)
    img = cv2.merge((img_b, img_g, img_r)) #merge adjusted channels
    del img_b
    del img_g
    del img_r
    return img

#given a list of samples, make two copies of each sample (one bluer, one greener) to simulate different pools + color attenuation
def colorAugment(images):
    out = []
    color_shift_intensity = int(255*0.1)
    for image in images:
        blue_img = make_bluer(image, color_shift_intensity)
        green_img = make_greener(image, color_shift_intensity)
        out.append(blue_img)
        out.append(green_img)
    return out

#given a single image and augmentation function, displays the image before and images after augmentation
def visualizeAugmentation(img, aug):
    #show original image
    cv2.imshow('og', img)
    cv2.waitKey(0)
    #show all augmented images
    for augmented in aug([(img, "")])[1:]:
        cv2.imshow('augmented',augmented[0])
        cv2.waitKey(0)

In [7]:
def get_file_names(source_folder):
    label_filenames = []
    img_filenames = [f for f in listdir(source_folder + '\\images') if isfile(join(source_folder + '\\images', f))]
    for img_filename in img_filenames:
        label_filenames.append(os.path.splitext(img_filename)[0] + ".txt")

    return np.array(img_filenames), np.array(label_filenames)

def split_file_names(images,labels,splits):
    perm = np.random.permutation(len(images))
    images = images[perm]
    labels = labels[perm]
    splits = [int(len(images)*s) for s in splits]
    train_images = images[:splits[0]]
    train_labels = labels[:splits[0]]
    val_images = images[splits[0]: splits[0] + splits[1]]
    val_labels = labels[splits[0]: splits[0] + splits[1]]
    test_images = images[splits[0] + splits[1]:]
    test_labels = labels[splits[0] + splits[1]:]
    return train_images, train_labels, val_images, val_labels, test_images, test_labels

def get_augs(img_filename,source_folder):
    img = cv2.imread(source_folder + '\\images\\' + img_filename)
    augs = [img]
    if(np.random.rand() < colorAugmentProb):
        augs = augs + colorAugment(augs)
    if(np.random.rand() < noiseAugmentProb):
        augs = augs + noiseAugment(augs)
    if(np.random.rand() < resolutionAugmentProb):
        augs = augs + resolutionAugment(augs)
    if(np.random.rand() < contrastAugmentProb):
        augs = augs + contrastAugment(augs)
    if(np.random.rand() < blurAugmentProb):
        augs = augs + blurAugment(augs)
    if(np.random.rand() < brightnessAugmentProb):
        augs = augs + brightnessAugment(augs)
    return augs

def do_augs_and_export(img_filenames,label_filenames,source_folder,output_folder):
    name_num = 1
    for (img_filename,label_filename) in zip(img_filenames,label_filenames):
        augs = get_augs(img_filename,source_folder)
        with open(source_folder + "\\labels\\" + label_filename) as f:
            #build array of bounding boxes (each line its own element)
            bounding_boxes = f.read()
        for aug in augs:
            cv2.imwrite(output_folder + '\\images\\img' + str(name_num) + '.png', aug)
            with open(output_folder + '\\labels\\img' + str(name_num) + '.txt',"w+") as f:
                f.write(bounding_boxes)
            name_num+=1

## Download dataset from Roboflow

In [8]:
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace(roboflow_workspace_name).project(roboflow_project_name)
version = project.version(roboflow_project_version)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to front-cam-felps-5 in yolov5pytorch:: 100%|██████████| 212/212 [00:00<00:00, 1724.89it/s]


In [ ]:
folder_name = "{}-{}".format(roboflow_project_name, roboflow_project_version)
roboflow_folder_absolute_path = os.path.abspath(folder_name)
data_folder_absolute_path = os.path.abspath("data")

print("Roboflow folder name:", folder_name)
print("Roboflow absolute path:", roboflow_folder_absolute_path)
print("Data absolute path:", data_folder_absolute_path)

!move "{roboflow_folder_absolute_path}\train\images\*.*" "{data_folder_absolute_path}\raw\images\"
!move "{roboflow_folder_absolute_path}\train\labels\*.*" "{data_folder_absolute_path}\raw\labels\"
!move "{roboflow_folder_absolute_path}\valid\images\*.*" "{data_folder_absolute_path}\raw\images\"
!move "{roboflow_folder_absolute_path}\valid\labels\*.*" "{data_folder_absolute_path}\raw\labels\"
!move "{roboflow_folder_absolute_path}\test\images\*.*" "{data_folder_absolute_path}\raw\images\"
!move "{roboflow_folder_absolute_path}\test\labels\*.*" "{data_folder_absolute_path}\raw\labels\"
!rmdir /s /q "{folder_name}"

## Augment data, split into train/test/val

In [10]:
out_folder = "data\\augmented"
in_folder = "data\\raw"

img_names, label_names = get_file_names(in_folder)
num_raw_samples = len(img_names)
train_images, train_labels, val_images, val_labels, test_images, test_labels = split_file_names(img_names,label_names,train_test_val_split)
do_augs_and_export(train_images,train_labels,in_folder,out_folder + "\\train")
do_augs_and_export(val_images,val_labels,in_folder,out_folder + "\\val")
do_augs_and_export(test_images,test_labels,in_folder,out_folder + "\\test")

augmented_train_img_names, _ = get_file_names(out_folder + "/train")
augmented_test_img_names, _ = get_file_names(out_folder + "/test")
augmented_val_img_names, _ = get_file_names(out_folder + "/val")

num_augmented_samples = len(augmented_train_img_names) + len(augmented_test_img_names) + len(augmented_val_img_names)

print("Augmentation completed. Went from {} raw samples to a total of {} after augmentation.".format(num_raw_samples, num_augmented_samples))

Pydantic serializer warnings:
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected


Augmentation completed. Went from 100 raw samples to a total of 1046 after augmentation.


# Check CUDA dependencies and start training

In [11]:
CUDA_setup_is_valid = int(torch.cuda.is_available() and torch.cuda.device_count())

print("Cuda is {}available!!!".format("" if CUDA_setup_is_valid else "NOT "))

Cuda is available!!!


In [16]:
!rmdir /s /q "runs"
!mkdir "runs"
!mkdir "runs/detect"

The system cannot find the file specified.


In [22]:
data_yaml_file_absolute_path = os.path.abspath("data.yaml")

model = YOLO("yolov8n.pt") #load a pretrained model

# Start the training process
while True:
    try:
        model.train(
            data=data_yaml_file_absolute_path,
            epochs=epoch_increments,
            # device=1, # --> it can automatically detect the device available
            batch=batch_size,
            degrees=degrees,
            flipud=flipud,
            fliplr=fliplr,
            perspective=max_perspective_change,
            translate=max_translate,
            scale=max_scale_change,
            mosaic=mosaic,
            mixup=mixup,
            pretrained=True,
            task='detect',
        )
        shutil.copyfile("runs\\detect\\train\\weights\\best.pt", "{}".format(model_save_filename))
    except RuntimeError as e:
        print(f"Caught a RuntimeError: {e}")
        break  # Break out of the loop if an error occurs to prevent infinite loop


New https://pypi.org/project/ultralytics/8.2.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.24 🚀 Python-3.11.9 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False